In [1]:
import time
import random
import datetime
import schedule
# dc_crawling.py 같은 폴더에 있다고 가정
#from dc_crawling import *from kafka impt KafkaProducer

In [2]:
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas# as pd
import datetime
from tqdm import tqdm

/Users/seomin/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

# 기존

In [17]:
#bootstrap_servers = ['localhost:9092','localhost:9093']  # kafka broker ip
bootstrap_servers = ['localhost:9092']
topicName = 'realtime_stock'
producer = KafkaProducer(bootstrap_servers = bootstrap_servers)


codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', 
'041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', 
'008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', 
'089890', '030350', '230980', '069410'}

now = datetime.datetime.now()
    
############ 
# {'code': '336260', 'time':'2022.01.07 14:43', 'price':'127,500'}
#############

headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

# ticker 처음부터 다 돌기
for code in codes:
    time = datetime.datetime(now.year, now.month, now.day)

    if now.weekday() == 5 or now.weekday() == 6:
        now -= datetime.timedelta(days = 1)
        continue
    
    result = list()
    pages = range(1, 13)
    for page in pages:
        url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, time.strftime("%Y%m%d")+"161122", page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        
        # result = for t, p in zip([i.text for i in dom.select('tr > td:nth-of-type(1) > span')], [i.text for i in dom.select('tr > td:nth-of-type(2) > span')])}
        for i in range(len(dom.select('tr > td:nth-of-type(2) > span'))):
            # result list에 dictionary 형태로 추가
            result.append({
                'code':code, 
                'date': time.strftime("%Y.%m.%d") + " " + dom.select('tr > td:nth-of-type(1) > span')[i].text,
                'price': dom.select('tr > td:nth-of-type(2) > span')[i].text
                }) 
            for value in result:
                producer.send(topicName, str(i).encode())
                tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                producer.send(topicName, tim.encode())
                print(value," - ")
            

# 날짜 왜 깎는지 잘 모르게씀..!           
now -= datetime.timedelta(days = 1)

# for value in ACODE.keys():
#     print(value)
#     DC_con = DC_crawling(value, 100)
    

#     for i in DC_con:
#         producer.send(topicName, str(i).encode())      
#         tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#         metadata = producer.send(topicName, tim.encode())
#         print(i," - ",metadata)

NameError: name 'KafkaProducer' is not defined

In [ ]:
# 1월 7일 -> 하루가 지나고 갱신 
#=> 이중 딕셔너리 자체를 초기화
# 스케쥴러로 9시 시작하니까  -> 그 자체에서 초기화 하던가
########## 메인함수에 초반에 초기화 시키는 부분   ==> #토픽 초기화하는 함수 부분 추가하고
# 이어서 append..!

#1 => 9:00 - 9:04
#2 => 9:05 - 9:09
#3 => 9:10 - 9:14

# Jan 09 수정

## 1차시도 : time 에 5를 더할 수 없는 상태

In [26]:
# #bootstrap_servers = ['localhost:9092','localhost:9093']  # kafka broker ip
# bootstrap_servers = ['localhost:9092']
# topicName = 'realtime_stock'
# producer = KafkaProducer(bootstrap_servers = bootstrap_servers)


codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', 
'041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', 
'008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', 
'089890', '030350', '230980', '069410'}

now = datetime.datetime.now()
    
############ 
# {'code': '336260', 'time':'2022.01.07 14:43', 'price':'127,500'}
#############

headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

# ticker 처음부터 다 돌기
for code in codes:
    date = datetime.datetime(now.year, now.month, now.day)

    if now.weekday() == 5 or now.weekday() == 6:
        now -= datetime.timedelta(days = 1)
        continue
    
    result = {}
    start = "09:00"        
    
    #result (dict)의 value 안에 들어갈 dict의 이름 : codestock
    codestock = {}
    
        
    pages = range(1, 13)
    for page in pages:
        url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, date.strftime("%Y%m%d")+"161122", page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        
        # result = for t, p in zip([i.text for i in dom.select('tr > td:nth-of-type(1) > span')], [i.text for i in dom.select('tr > td:nth-of-type(2) > span')])}
        for i in range(len(dom.select('tr > td:nth-of-type(2) > span'))):
            # result list에 dictionary 형태로 추가
            time = dom.select('tr > td:nth-of-type(1) > span')[i].text
            j = 1
            schedule.every().minutes.at("09:00").do(
            while start == "16:00":

                if (start <= time) and (time < start+5):
                    codestock.keys[j] = codestock.keys[j].append({
                        'code':code
                    })
                    codestock.values[j]=codestock.values[j].append({
                        'date': date.strftime("%Y.%m.%d") + " " + time,
                        'price': dom.select('tr > td:nth-of-type(2) > span')[i].text
                        }) 
                    result.values[j] = codestock.items[j]
                    j += 1
                    continue
                    
                else:
                    print(start)
                    
                    break
            )  
#             for value in result:
#                 producer.send(topicName, str(i).encode())
#                 tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#                 producer.send(topicName, tim.encode())
#                 print(value," - ")
            

# 날짜 왜 깎는지 잘 모르게씀..!           
now -= datetime.timedelta(days = 1)

## 예진언니 코드 인용 ver (미완)

In [40]:
#예진언니 코드 인용 ver

codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', 
'041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', 
'008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', 
'089890', '030350', '230980', '069410'}

now = datetime.datetime.now()
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

# ticker 처음부터 다 돌기
for code in codes:
    #date = datetime.datetime(now.year, now.month, now.day)

    #if now.weekday() == 5 or now.weekday() == 6:
#         now -= datetime.timedelta(days = 1)
#         continue
    time = datetime.datetime(2022,1,9,9,0)
    result = {}
    start = "09:00" 
    
    #result (dict)의 value 안에 들어갈 dict의 이름 : codestock
    codestock = {}
    

    pages = range(1, 13)
        
    for page in pages:
        url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, date.strftime("%Y%m%d")+"161122", page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')

        # result = for t, p in zip([i.text for i in dom.select('tr > td:nth-of-type(1) > span')], [i.text for i in dom.select('tr > td:nth-of-type(2) > span')])}
        for i in range(len(dom.select('tr > td:nth-of-type(2) > span'))): # 한 종목의 주가
            # result list에 dictionary 형태로 추가
            time = dom.select('tr > td:nth-of-type(1) > span')[i].text
            time = datetime.datetime.strptime(time, "%H:%M")

            j = 1
            while time < datetime.datetime.strptime("16:05", "%H:%M"):
                
                (date.strftime("%Y.%m.%d") + " " + time, dom.select('tr > td:nth-of-type(2) > span')[i].text)
                
                
                result.values[j] = codestock.items[j]
                time += datetime.timedelta(minutes=5)  
                j += 1

            break


TypeError: 'builtin_function_or_method' object is not subscriptable

In [ ]:
# RESULT = dict{i : [] for i in range()}
i = 1
time = datetime.datetime(2022,1,9,9,0)
while time.strftime('%H:%M') < "16:05":
    
    time += datetime.timedelta(minutes=5)    
    i += 1

## addstock 함수 만든 ver

In [25]:
def addstock(result):
    
    codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', 
    '041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', 
    '008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', 
    '089890', '030350', '230980', '069410'}
    
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}


    now = datetime.datetime.now()
    
    for code in codes:
        date = datetime.datetime(now.year, now.month, now.day)

        if now.weekday() == 5 or now.weekday() == 6:
            now -= datetime.timedelta(days = 1)
            continue
    
        result = {}
        start = "09:00"        

        #result (dict)의 value 안에 들어갈 dict의 이름 : codestock
        codestock = {}


        pages = range(1, 13)
        for page in pages:
            url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, date.strftime("%Y%m%d")+"161122", page)
            resp = download(url, headers = headers)
            dom = BeautifulSoup(resp.content, 'lxml')

                # result = for t, p in zip([i.text for i in dom.select('tr > td:nth-of-type(1) > span')], [i.text for i in dom.select('tr > td:nth-of-type(2) > span')])}
            for i in range(len(dom.select('tr > td:nth-of-type(2) > span'))):
                    # result list에 dictionary 형태로 추가
                time = dom.select('tr > td:nth-of-type(1) > span')[i].text


                j=1

                while start == "16:00":

                    codestock.keys[j] = codestock.keys[j].append({
                                        'code':code
                                    })
                    codestock.values[j]=codestock.values[j].append({
                                        'date': date.strftime("%Y.%m.%d") + " " + time,
                                        'price': dom.select('tr > td:nth-of-type(2) > span')[i].text
                                        }) 

                    result.values[j] = codestock.items[j]
                    j += 1      


                return result

In [29]:
schedule.every(5).minutes.do(addstock)
        

# 날짜 왜 깎는지 잘 모르게씀..!           
#now -= datetime.timedelta(days = 1)

Every 5 minutes do addstock() (last run: [never], next run: 2022-01-09 21:12:24)